In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import TruncatedSVD
import cluster_class
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import train_test_split
import keras.utils


,sentence,author
0,"[eugenie, grandet, by, honore, de, balzac, tra...","Balzac, Honoré de"
1,"[may, your, name,, that, of, one, whose, portr...","Balzac, Honoré de"
2,"[de, balzac.]","Balzac, Honoré de"
3,"[eugenie, grandet, i, there, are, houses, in, ...","Balzac, Honoré de"
4,"[within, these, houses, there, is,, perhaps,, ...","Balzac, Honoré de"


In [3]:
f = open("../clean_text_with_punc.pickle","rb")
df = pickle.load(f)
f.close()

# turn list into string
df['sentence'] = df['sentence'].apply(lambda x:' '.join(x))
df.head()

,sentence,author
0,eugenie grandet by honore de balzac translated...,"Balzac, Honoré de"
1,"may your name, that of one whose portrait is t...","Balzac, Honoré de"
2,de balzac.,"Balzac, Honoré de"
3,eugenie grandet i there are houses in certain ...,"Balzac, Honoré de"
4,"within these houses there is, perhaps, the sil...","Balzac, Honoré de"


## Process label

In [15]:
# turn author binerize
#[0,0,0,1,0,0,0,0,.....]
# author_list = list(set(df['author']))
# label = []
# for i in df['author']:
#     label.append([1 if author_list[j] == i else 0 for j in range(len(author_list))])

# [0,15,13,7,3,...]
author_list = list(set(df['author']))
label = []
for i in df['author']:
    label.append(author_list.index(i))

In [8]:
X = []
for i in df['sentence']:
    X.append(model.infer_vector(i.split()))
X[0]

array([ 0.04289634, -0.00835135, -0.04898711,  0.03799013,  0.0365734 ,
        0.00756317,  0.04480249,  0.0297677 , -0.00095395, -0.01895812],
      dtype=float32)

In [36]:
f = open("word_model_Doc2Vec_raw.pickle","rb")
model = pickle.load(f)
f.close()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

In [32]:
lg = LogisticRegression(max_iter=500).fit(X_train, y_train)

## Predict probability

In [33]:
scores = lg.predict_proba(X_test)
pred = scores.argmax(axis=1)

In [34]:
right, wrong = 0, 0
for i,j in zip(pred, y_test):
    if i == j:
        right += 1
    else:
        wrong += 1
right/(wrong+right)

0.09934640522875816